In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from tqdm import tqdm
from IPython.display import clear_output
from time import sleep
import re

In [2]:
years = range(1971, 2023)
seasons = [str(year - 1) + "-" + str(year)[-2:] for year in years]
numeric = pd.DataFrame({"year": years, "season": seasons})

In [3]:
standings = pd.DataFrame()
timeout = 5
for season in tqdm(numeric.season):
    print(season)
    driver = webdriver.Chrome(executable_path='C:/Program Files/chromedriver.exe')
    url = f"https://www.nba.com/standings?GroupBy=conf&Season={season}&Section=overall"
    driver.get(url)
    element_present = EC.presence_of_element_located((By.CLASS_NAME, 'h5'))
    WebDriverWait(driver, timeout).until(element_present)
    html = driver.page_source
    tables = pd.read_html(html)
    data1 = tables[0]
    data2 = tables[1]
    data = pd.concat([data1, data2], axis=0, ignore_index=True)
    data.TEAM = data.TEAM.str.extract("([A-Za-z].*)")
    data.TEAM = data.TEAM.apply(lambda x: x[:-4])
    data['SEASON'] = season
    standings = pd.concat([standings, data], axis=0, ignore_index=True)
    clear_output(wait=True)

100%|██████████| 52/52 [06:00<00:00,  6.93s/it]


FileNotFoundError: [Errno 2] No such file or directory: 'data/standingsNEW.csv'

---

In [170]:
standings = pd.read_csv('../data/standingsNEW.csv')

In [4]:
def get_names(row):
    for i, v in enumerate(row):
        if v == '-':
            return row[:i].strip()
    return row

In [5]:
standings.TEAM = standings.TEAM.apply(get_names)

In [6]:
def fix_team_names(row):
    if row in ["New Jersey Nets", "New York Nets"]:
        return "Brooklyn Nets"
    if row in ["Washington Bullets", "Baltimore Bullets", "Capital Bullets"]:
        return "Washington Wizards"
    if row in ["LA Clippers", "San Diego Clippers", "Buffalo Braves"]:
        return "Los Angeles Clippers"
    if row in ["Kansas City Kings", "Cincinnati Royals", "Kansas City"]:
        return "Sacramento Kings"
    if row == "Seattle SuperSonics":
        return "Oklahoma City Thunder"
    if row in ["New Orleans/Oklahoma City Hornets",
                     "New Orleans Hornets"]:
        return "New Orleans Pelicans"
    if row == "Charlotte Bobcats":
        return "Charlotte Hornets"
    if row == "Vancouver Grizzlies":
        return "Memphis Grizzlies"
    if row == "San Francisco Warriors":
        return "Golden State Warriors"
    if row == "San Diego Rockets":
        return "Houston Rockets"
    if row == "New Orleans Jazz":
        return "Utah Jazz"
    return row

In [7]:
standings.TEAM = standings.TEAM.apply(fix_team_names)

In [178]:
# standings.drop(['LAST 10','STREAK'],axis=1, inplace=True)

In [8]:
standings.to_csv('standingsCleaned.csv', index=False)